necessary packages

In [982]:
import numpy as np
from ipywidgets import interact
import plotly.graph_objects as go
from scipy.special import gamma
from scipy.integrate import quad as integrate

define function for updating the layout of the figure:
* updates the layout of a plotly figure with predefined settings
* the **title** dict includes all title properties: **'x'** specifies the position of the title along the x-axis, **'xanchor'** specifies the anchor point of the title along the x-axis

In [983]:
def update_layout_of_graph(fig: go.Figure, title: str = 'Plot') -> go.Figure:
    fig.update_layout(
        width=1000,
        height=500,
        autosize=False,
        plot_bgcolor='black',
        title={'text': title, 'x': 0.5, 'xanchor': 'center'},  # dict includes all title properties; specifies the position of the title along the x-axis
        xaxis_title='t / ms',
        yaxis_title='I / A',
        legend=dict(yanchor="top", y=0.5, xanchor="right", x=0.95),
        xaxis=dict(showline=True, linewidth=1, linecolor='black'),
        yaxis=dict(showline=True, linewidth=1, linecolor='black')
    )
    return fig

define function for creating a line plot:
* creates a Plotly scatter trace representing a line plot
* **x_lines** x-coordinates of the points on the line with an empty default array
* **y_lines** y-coordinates of the points on the line with an empty default array

In [984]:
def line_plot(x_lines: np.array = np.array([]), y_lines: np.array = np.array([]), title: str = 'predicted function',
show_legend: bool = True, visible: bool = True, ) -> go.Scatter:

    scatter = go.Scatter(x=x_lines, y=y_lines, line=dict(color="orange", width=3), name=title, showlegend= show_legend, visible=visible)
    return scatter

define function for a scatter plot:
* creates a scatter plot with Plotly
* **x_dots** is an array of x-coordinates for the data points in the scatter plot with an empty default array
* **y_dots** is an array of y-coordinates for the data points in the scatter plot with an empty default array

In [985]:
def scatter_plot(visible: bool = True, x_dots: np.array = np.array([]), y_dots: np.array = np.array([]), name_dots: str = 'observed points',
show_legend: bool = True, color: str = 'red') -> go.Scatter:

    scatter = go.Scatter(x=x_dots, y=y_dots, visible=visible, mode='markers', name=name_dots, marker=dict(color=color, size=8), showlegend=show_legend)
    return scatter

define function for plotting the uncertainty area:
* generates an area plot to represent the uncertainty using the Plotly
* **x_lines** an array of x-coordinates for the lines outlining the area plot with an empty default array
* **y_lower** an array of y-coordinates representing the lower boundary of the uncertainty area plot with an empty default array
* **y_upper** an array of y-coordinates representing the upper boundary of the uncertainty area plot with an empty default array

In [986]:
def uncertainty_area_plot(visible: bool = True, x_lines: np.array = np.array([]), y_lower: np.array = np.array([]), y_upper: np.array = np.array([]), name: str = 'mean plus/minus standard deviation') -> go.Scatter:

    scatter = go.Scatter(x=np.concatenate((x_lines, x_lines[::-1])), y=np.concatenate((y_upper, y_lower[::-1])), visible=visible, fill='toself', fillcolor='rgba(189, 195, 199, 0.5)', line=dict(color='rgba(200, 200, 200, 0)'), hoverinfo='skip', showlegend=True, name=name)
    return scatter

define squared exponential kernel:
* **variable1** & **variable2** correspond to t & t'
* **sigma** describes the average distance away from the function mean and l determines the reach of influence on neighbors
* **sigma** & **length** is assigned the float standard value 1
* **variable_1** & **variable_2** expect an array

In [987]:
class SquaredExponentialKernel:
    def __init__(self, sigma: float = 1, length: float = 1, alpha: float = 0.75, beta: float = 0.75):
        self.sigma = sigma
        self.length = length
        self.alpha = alpha
        self.beta = beta

    def __call__(self, tau_1: np.array, tau_2: np.array):
        def kernel(variable_1, variable_2):
            k_SE = float(self.sigma**2 * np.exp(-(np.linalg.norm(variable_1 - variable_2)**2) /(2 * self.length**2)))
            return k_SE
        return kernel(tau_1, tau_2)

visualize the squared exponential kernel:
* only for testing the squared exponential kernel and monitor the effect of changing the parameters
* **@interact** generates a user interface which allows for an interactive regulation of the parameters length, sigma and variable_2 in the defined ranges
* the function **update** is called whenever the values of the sliders are changed
* **batch_update()** ensures that all updates to the figure within the block are batched together

In [988]:
x_lines = np.arange(-10, 10, 0.1)
kernel = SquaredExponentialKernel()  # initialize an instance of the class SquaredExponentialKernel, length and sigma have already standard values
data = line_plot(x_lines=x_lines, y_lines=np.array([kernel(x, 0) for x in x_lines]))  # generates the data for the line distribution

fig = go.FigureWidget(data)  # generates an interactive plot-widget with the passed data
fig = update_layout_of_graph(fig, title='squared exponential kernel')

@interact(length=(0.1, 3, 0.1), sigma=(0.1, 3, 0.1), variable_2=(-10, 10, 0.1))
def update(length=1, sigma=1, variable_2=0):
    with fig.batch_update():
        kernel = SquaredExponentialKernel(sigma=sigma, length=length)  # a new instance of the class SquaredExponentialKernel with the updated length and sigma value is generated
        fig.data[0].y = np.array([kernel(x, variable_2) for x in x_lines])  # updates the y-values plot data, data can contain multiply scatter plots
        # with data[0].y the y-coordinates of the first scatter plot are accessed

fig

interactive(children=(FloatSlider(value=1.0, description='length', max=3.0, min=0.1), FloatSlider(value=1.0, d…

FigureWidget({
    'data': [{'line': {'color': 'orange', 'width': 3},
              'name': 'predicted function',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'cebee62f-f57f-4b23-80a3-9714cfe67b28',
              'visible': True,
              'x': array([-1.00000000e+01, -9.90000000e+00, -9.80000000e+00, -9.70000000e+00,
                          -9.60000000e+00, -9.50000000e+00, -9.40000000e+00, -9.30000000e+00,
                          -9.20000000e+00, -9.10000000e+00, -9.00000000e+00, -8.90000000e+00,
                          -8.80000000e+00, -8.70000000e+00, -8.60000000e+00, -8.50000000e+00,
                          -8.40000000e+00, -8.30000000e+00, -8.20000000e+00, -8.10000000e+00,
                          -8.00000000e+00, -7.90000000e+00, -7.80000000e+00, -7.70000000e+00,
                          -7.60000000e+00, -7.50000000e+00, -7.40000000e+00, -7.30000000e+00,
                          -7.20000000e+00, -7.10000000e+00, -7.000000

functions for creating the covariance matrix, each of them were implemented separately in such a way as to make them comprehensible with the notes, which is why they are referenced:
* **val_1** & **val_2** are equal to a certain element of the arrays **t** & **t'**
* **alpha**, **R** & **C** are set to a specific value

In [989]:
def k_fu(val_1, val_2, cov_func, R=0.87, C=330e-6, L=1e-3):
    alpha = cov_func.alpha
    beta = cov_func.beta
    l = cov_func.length

    c = R * C * cov_func(val_1, val_2)

    def integrand_a(tau):
        a = -1 / (gamma(1+alpha) * l**2)
        return a * (tau - val_2) * (val_1 - tau)**alpha * cov_func(tau, val_2)

    def integrand_b(tau):
        b = -1 / (gamma(1+alpha+beta) * l**2)
        return b * (tau - val_2) * (val_1 - tau)**(alpha+beta) * cov_func(tau, val_2)

    integral_val_a, _ = integrate(integrand_a, 0, val_1, epsabs=1e-9, epsrel=1e-9, limit=10000)
    integral_val_b, _ = integrate(integrand_b, 0, val_1, epsabs=1e-9, epsrel=1e-9, limit=10000)
    return integral_val_a + R / L * integral_val_b + c

In [990]:
def k_uf(val_1, val_2, cov_func, R=0.87, C=330e-6, L=1e-3):
    alpha = cov_func.alpha
    beta = cov_func.beta
    l = cov_func.length

    c = R * C * cov_func(val_1, val_2)

    def integrand_a(tau):
        a = 1 / (gamma(1+alpha) * l**2)
        return a * (val_1 - tau) * (val_2 - tau)**alpha * cov_func(val_1, tau)

    def integrand_b(tau):
        b = 1 / (gamma(1+alpha+beta) * l**2)
        return b * (val_1 - tau) * (val_2 - tau)**(alpha+beta) * cov_func(val_1, tau)

    integral_val_a, _ = integrate(integrand_a, 0, val_2, epsabs=1e-9, epsrel=1e-9, limit=10000)
    integral_val_b, _ = integrate(integrand_b, 0, val_2, epsabs=1e-9, epsrel=1e-9, limit=10000)
    return integral_val_a + R / L * integral_val_b + c

In [991]:
def k_uu(val_1, val_2, cov_func):
    return cov_func(val_1, val_2)

In [992]:
def k_ff(val_1, val_2, cov_func, R=0.87, C=330e-6, L=1e-3):
    alpha = cov_func.alpha
    beta = cov_func.beta
    l = cov_func.length

    # simple expressions, see notes:
    a = R * C * k_uf(val_1, val_2, cov_func, R, C, L)
    b = R * C * (k_fu(val_1, val_2, cov_func, R, C, L) - R * C * k_uu(val_1, val_2, cov_func))

    # more complex integral c, see notes:
    def c():
        def A(val_1, val_2, tau):
            a = 1/gamma(1+alpha) * (val_2 - tau)**alpha * 1/l**2 * (val_1 - tau)
            return a * cov_func(val_1, tau)

        def B(val_1, val_2):
            integrand = lambda tau: A(val_1, val_2, tau)
            integral_val, _ = integrate(integrand, 0, val_2, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        def derivative_B(tau, val_2, h= 0.01*np.sqrt(2e-53)):
            def B_tau(tau):
                return B(tau, val_2)
            dB = (B_tau(tau + h) - B_tau(tau - h)) / (2*h)
            return dB

        def double_integral(val_1, val_2):
            integrand = lambda tau: 1/gamma(1 + alpha) * (val_1 - tau)**alpha * derivative_B(tau, val_2)
            integral_val, _ = integrate(integrand, 0, val_1, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        c = double_integral(val_1, val_2)
        return c

    # more complex integral d, see notes:
    def d():
        def A(val_1, val_2, tau):
            a = R / (L * gamma(1+alpha+beta)) * (val_2 - tau)**(alpha+beta) * 1/l**2 * (val_1 - tau)
            return a * cov_func(val_1, tau)

        def B(val_1, val_2):
            integrand = lambda tau: A(val_1, val_2, tau)
            integral_val, _ = integrate(integrand, 0, val_2, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        def derivative_B(tau, val_2, h= 0.01*np.sqrt(2e-53)):
            def B_tau(tau):
                return B(tau, val_2)
            dB = (B_tau(tau + h) - B_tau(tau - h)) / (2*h)
            return dB

        def double_integral(val_1, val_2):
            integrand = lambda tau: 1/gamma(1 + alpha) * (val_1 - tau)**alpha * derivative_B(tau, val_2)
            integral_val, _ = integrate(integrand, 0, val_1, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        d = double_integral(val_1, val_2)
        return d

    # more complex integral e, see notes:
    def e():
        def A(val_1, val_2, tau):
            a = 1/gamma(1+alpha) * (val_2 - tau)**alpha * 1/l**2 * (val_1 - tau)
            return a * cov_func(val_1, tau)

        def B(val_1, val_2):
            integrand = lambda tau: A(val_1, val_2, tau)
            integral_val, _ = integrate(integrand, 0, val_2, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        def derivative_B(tau, val_2, h= 0.01*np.sqrt(2e-53)):
            def B_tau(tau):
                return B(tau, val_2)
            dB = (B_tau(tau + h) - B_tau(tau - h)) / (2*h)
            return dB

        def double_integral(val_1, val_2):
            integrand = lambda tau: 1 / gamma(1 + alpha + beta) * (val_1 - tau)**(alpha + beta) * derivative_B(tau, val_2)
            integral_val, _ = integrate(integrand, 0, val_1, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        e = R / L * double_integral(val_1, val_2)
        return e

    # more complex integral f, see notes:
    def f():
        def A(val_1, val_2, tau):
            a = R / (L * gamma(1 + alpha + beta)) * (val_2 - tau)**(alpha + beta) * 1/l**2 * (val_1 - tau)
            return a * cov_func(val_1, tau)

        def B(val_1, val_2):
            integrand = lambda tau: A(val_1, val_2, tau)
            integral_val, _ = integrate(integrand, 0, val_2, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        def derivative_B(tau, val_2, h= 0.01*np.sqrt(2e-53)):
            def B_tau(tau):
                return B(tau, val_2)
            dB = (B_tau(tau + h) - B_tau(tau - h)) / (2*h)
            return dB

        def double_integral(val_1, val_2):
            integrand = lambda tau: 1 / gamma(1 + alpha + beta) * (val_1 - tau)**(alpha + beta) * derivative_B(tau, val_2)
            integral_val, _ = integrate(integrand, 0, val_1, epsabs=1e-8, epsrel=1e-8, limit=10000)
            return integral_val

        d = R / L * double_integral(val_1, val_2)
        return d

    return a + b + c() + d() + e() + f()

testing the implemented functions above:

In [993]:
val_1 = 0.1
val_2 = 0.1
x = np.arange(0.01, 3, 0.01)
cov_func = SquaredExponentialKernel()

for i in range(len(x)):
    kuf = k_uf(x[i], x[i], cov_func)
    kfu = k_fu(x[i], x[i], cov_func)
    kuu = k_uu(x[i], x[i], cov_func)
    kff = k_ff(x[i], x[i], cov_func)
    print('kuf = ', kuf, 'kuu = ', kuu, 'kfu = ', kfu, 'kff = ', kff)

kuf =  0.0003070494106657661 kuu =  1.0 kfu =  0.0003070494106657661 kff =  9.388136160428288e-08
kuf =  0.0005070424443475588 kuu =  1.0 kfu =  0.0005070424443475588 kff =  2.0871736154436826e-07
kuf =  0.001186970645058045 kuu =  1.0 kfu =  0.001186970645058045 kff =  5.991321343923294e-07
kuf =  0.002735931327708859 kuu =  1.0 kfu =  0.002735931327708859 kff =  1.4885453583704266e-06
kuf =  0.005613947511904068 kuu =  1.0 kfu =  0.005613947511904068 kff =  3.141102251335316e-06
kuf =  0.010341646737587954 kuu =  1.0 kfu =  0.010341646737587954 kff =  5.855747146723003e-06
kuf =  0.017493208442374654 kuu =  1.0 kfu =  0.017493208442374654 kff =  9.962173877611525e-06
kuf =  0.02769105925971123 kuu =  1.0 kfu =  0.02769105925971123 kff =  1.581777981692619e-05
kuf =  0.04160163276631617 kuu =  1.0 kfu =  0.04160163276631617 kff =  2.3805231124418744e-05
kuf =  0.05993183223639681 kuu =  1.0 kfu =  0.05993183223639681 kff =  3.433043166013905e-05
kuf =  0.08342598450086934 kuu =  1.0 k

a function for creating the sub and covariance matrix as described in the notes:
* works only correctly for squared matrices [n1 x n2] where n1 = n2

In [994]:
def create_sub_matrix(variable_1, variable_2, specified_cov_function, cov_function):
    n1 = len(variable_1)
    n2 = len(variable_2)
    sub_matrix = np.zeros((n1, n2))

    for i, val_1 in enumerate(variable_1):
        for j, val_2 in enumerate(variable_2):
            sub_matrix[i, j] = specified_cov_function(val_1, val_2, cov_function)

    return sub_matrix

a function for creating the covariance matrix as described in the notes:
* works only correctly for squared matrices [2*n1 x 2*n2] where n1 = n2

In [995]:
def create_cov_matrix(variable_1, variable_2, cov_function):
    n1 = len(variable_1)
    n2 = len(variable_2)
    cov = np.zeros((2*n1, 2*n2))

    kuu_sub_matrix = create_sub_matrix(variable_1, variable_2, k_uu, cov_function)
    kuf_sub_matrix = create_sub_matrix(variable_1, variable_2, k_uf, cov_function)
    kfu_sub_matrix = create_sub_matrix(variable_1, variable_2, k_fu, cov_function)
    kff_sub_matrix = create_sub_matrix(variable_1, variable_2, k_ff, cov_function)

    for i in range(2*n1):
        for j in range(2*n2):

            if i <= n1 - 1 and j <= n2 - 1:
                #print('kuu')
                cov[i, j] = kuu_sub_matrix[i, j]
            elif i <= n1 - 1 and n2 <= j <= 2*n2 - 1:
                #print('kuf')
                cov[i, j] = kuf_sub_matrix[i, j - n2]
            elif n1 <= i <= 2*n1 - 1 and j <= n2 - 1:
                #print('kfu')
                cov[i, j] = kfu_sub_matrix[i - n1, j]
            else:
                #print('kff')
                cov[i, j] = kff_sub_matrix[i - n1, j - n2]

    return cov

testing the implemented functions for creating the covariance matrix above

In [996]:
variable_1 = [1, 1, 1, 1]
variable_2 = [1, 1, 1, 1]
#variable_1 = np.array([0.3, 0.8, 1.4, 1.8, 2])
#variable_2 = np.array([0.3, 0.8, 1.4, 1.8, 2])

cov_func = SquaredExponentialKernel()
cov = create_cov_matrix(variable_1, variable_2, cov_func)

print('covariance matrix = ', cov)

covariance matrix =  [[1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 1.00000000e+00
  1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02]
 [1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02
  7.88951653e-02 7.88951653e-02 7.88951653e-02 7.88951653e-02]
 [1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02
  7.88951653e-02 7.88951653e-02 7.88951653e-02 7.88951653e-02]
 [1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02
  7.88951653e-02 7.88951653e-02 7.88951653e-02 7.88951653e-02]
 [1.37400292e+02 1.37400292e+02 1.37400292e+02 1.37400292e+02
  7.88951653e-02 7.88951653e-02 7.88951653

define function for calculating q_u transposed


In [997]:
def q_u_transposed(variable_1, variable_2, cov_function):
    n1 = len(variable_1)
    n2 = len(variable_2)
    q_u_trans = np.zeros((n1, 2*n2))

    kuu_sub_matrix = create_sub_matrix(variable_1, variable_2, k_uu, cov_function)
    kuf_sub_matrix = create_sub_matrix(variable_1, variable_2, k_uf, cov_function)

    for i in range(n1):
        for j in range(2*n2):

            if j <= n2 - 1:
                q_u_trans[i, j] = kuu_sub_matrix[i, j]
            else:
                q_u_trans[i, j] = kuf_sub_matrix[i, j - n2]

    return q_u_trans

In [998]:
def q_f_transposed(variable_1, variable_2, cov_function):
    n1 = len(variable_1)
    n2 = len(variable_2)
    q_f_trans = np.zeros((n1, 2*n2))

    kfu_sub_matrix = create_sub_matrix(variable_1, variable_2, k_fu, cov_function)
    kff_sub_matrix = create_sub_matrix(variable_1, variable_2, k_ff, cov_function)

    for i in range(n1):
        for j in range(2*n2):

            if j <= n2 - 1:
                q_f_trans[i, j] = kfu_sub_matrix[i, j]
            else:
                q_f_trans[i, j] = kff_sub_matrix[i, j - n2]

    return q_f_trans

testing if the function q_u_transposed works properly

In [999]:
variable_1 = [1, 1, 1, 1]
variable_2 = [1, 2, 1, 1]
cov_func = SquaredExponentialKernel()

q_u_trans = q_u_transposed(variable_1, variable_2, cov_func)
print('q_u transposed = ', q_u_trans)

q_u transposed =  [[  1.           0.60653066   1.           1.         137.40029216
  475.66522877 137.40029216 137.40029216]
 [  1.           0.60653066   1.           1.         137.40029216
  475.66522877 137.40029216 137.40029216]
 [  1.           0.60653066   1.           1.         137.40029216
  475.66522877 137.40029216 137.40029216]
 [  1.           0.60653066   1.           1.         137.40029216
  475.66522877 137.40029216 137.40029216]]


new training method: page 113 in GP for ML

In [1000]:
def log_marginal_likelihood(data_x, data_y, cov_func, noise):
    n = len(data_x)
    y = np.concatenate([data_y, data_y])
    y_transposed = y.T
    K = create_cov_matrix(data_x, data_x, cov_func) + noise * np.identity(2*n)
    K_inverse = np.linalg.inv(K)

    a = n/2 * np.log(2*np.pi)  # normalization constant
    b = 1/2 * np.log(np.linalg.norm(K))  # complexity penalty
    c = 1/2 * np.dot(y, np.dot(K_inverse, y_transposed))  # involving observed targets
    log_ml = c + b + a  # is actually the negative log marginal likelihood
    return log_ml

sigma_vals = np.arange(0.2, 1, 0.05)
length_vals = np.arange(0.1, 2.5, 0.05)
memory = {'sigma': 1, 'length': 1, 'negative_log_marginal_likelihood': 1000}

data_x = np.array([0.06078808034380694, 0.6117048213974552, 1.3795619642545989, 2.002887857111742, 2.733914642826028])
data_y = np.array([-1.783753, -0.1626748, 0.1480789, 0.1019231, 0.0457059])

for sigma_index, sigma_val in enumerate(sigma_vals):
    for l_index, length_val in enumerate(length_vals):
        cov_func = SquaredExponentialKernel(sigma=sigma_val, length=length_val)
        negative_log_marginal_likelihood = log_marginal_likelihood(data_x, data_y, cov_func, noise = 0.01)
        print('likelihood = ', negative_log_marginal_likelihood)

        if negative_log_marginal_likelihood < memory['negative_log_marginal_likelihood']:
            memory['length'] = length_val
            memory['sigma'] = sigma_val
            memory['negative_log_marginal_likelihood'] = negative_log_marginal_likelihood

        print('obtained sigma and l values that minimize the negative log marginal likelihood: ', 'sigma = ', memory['sigma'], 'length = ', memory['length'])

likelihood =  166.62032708949846
obtained sigma and l values that minimize the negative log marginal likelihood:  sigma =  0.2 length =  0.1


KeyboardInterrupt: 

In [981]:
cov_func = SquaredExponentialKernel(sigma=0.375, length=2.5)
negative_log_marginal_likelihood = log_marginal_likelihood(data_x, data_y, cov_func, noise = 0.01)
print(negative_log_marginal_likelihood)

163.2663783728347


define class GPR:
* **data_x** is the input of the observed points
* **data_y** is the output of the observed points
* **noise** is the noise in the observed data
* **memory** is a variable where the mean, the covariance matrix and the variance is stored
* **3e-7** add this value to the noise in order to ensure stability when inverting the covariance matrix

In [1001]:
class GPR:
    def __init__(self, data_x, data_y, cov_func= SquaredExponentialKernel(sigma=memory['sigma'], length=memory['length']), noise=0.01):
        print('sigma memory = ', memory['sigma'])
        print('length memory = ', memory['length'])
        self.data_x = data_x
        self.data_y = np.concatenate([data_y, data_y])
        self.cov_func = cov_func
        self.noise = noise
        self.memory = None

        n = len(data_x)
        self.inv_cov_matrix_of_input_data = np.linalg.inv(create_cov_matrix(data_x, data_x, cov_func) + (noise + 3e-7)**2 * np.identity(2*n))

    def predict(self, at_values):
        q_u_trans = q_u_transposed(at_values, self.data_x, self.cov_func)
        q_f_trans = q_f_transposed(at_values, self.data_x, self.cov_func)

        # methode paper:
        mean_at_values_u = np.dot(q_u_trans, np.dot(self.data_y, self.inv_cov_matrix_of_input_data.T).T).flatten()
        #mean_at_values_f = np.dot(q_f_trans, np.dot(self.data_y, self.inv_cov_matrix_of_input_data.T).T).flatten()

        covariance_matrix_u = create_sub_matrix(at_values, at_values, k_uu, self.cov_func) - np.dot(q_u_trans, np.dot(self.inv_cov_matrix_of_input_data, q_u_trans.T))
        #covariance_matrix_f = create_sub_matrix(at_values, at_values, k_ff, self.cov_func) - np.dot(q_f_trans, np.dot(self.inv_cov_matrix_of_input_data, q_f_trans.T))

        variance_u = np.abs(np.diag(covariance_matrix_u))
        #variance_f = np.abs(np.diag(covariance_matrix_f))

        self.memory = {'mean': mean_at_values_u, 'covariance_matrix':covariance_matrix_u, 'variance': variance_u}

        return mean_at_values_u

defining function for plotting the GPR:
* **data_x** is a list of observed input data points
* **data_y** is a list of observed output data points
* **x** is equivalent to **at_values** in the GPR, the values we want to do predictions on

In [1002]:
def plot_GPR(data_x, data_y, model, x, data_x_mv, data_y_mv, visible=True) -> list:
    mean = model.predict(x)
    standard_deviation = np.sqrt(model.memory['variance'])
    data = []

    for i in range(1, 4):
        data.append(uncertainty_area_plot(x_lines=x, y_lower=mean -i * standard_deviation, y_upper=mean + i*standard_deviation, name=f'mean plus/minus {i}*standard deviation', visible=visible))

    data.append(scatter_plot(x_dots=data_x_mv, y_dots=data_y_mv, visible=visible, color='blue'))
    data.append(scatter_plot(x_dots=data_x, y_dots=data_y, visible=visible, color='red'))
    data.append(line_plot(x_lines=x, y_lines=mean, visible=visible))
    return data

initializing the training data

In [1003]:
x_values = np.array([0.06078808034380694, 0.6117048213974552, 1.3795619642545989, 2.002887857111742, 2.733914642826028])
y_values = np.array([-1.783753, -0.1626748, 0.1480789, 0.1019231, 0.0457059])


x_values_mv = np.array([7.812499952519758e-08, 0.02026274553126866, 0.040525412937537794, 0.06078808034380694, 0.08105074775007608, 0.1013134151563452, 0.1215760825626144, 0.14183874996888338, 0.1513253571117406, 0.1608119642545978, 0.17029857139745488, 0.17978517854031212, 0.1892717856831692, 0.1987583928260264, 0.20824499996888352, 0.21215124996888352, 0.21605749996888352, 0.21996374996888352, 0.2277762499688835, 0.23949499996888352, 0.2512137499688835, 0.27074499996888346, 0.2902762499688835, 0.3098074999688836, 0.32933874996888357, 0.3488699999688837, 0.36840124996888374, 0.38793249996888357, 0.41416017854031223, 0.4403878571117408, 0.4666155356831694, 0.492843214254598, 0.5190708928260266, 0.5452985713974552, 0.5715262499688838, 0.5916155356831695, 0.6117048213974552, 0.631794107111741, 0.6518833928260267, 0.6719726785403124, 0.6920619642545982, 0.7121512499688839, 0.721637857111741, 0.7311244642545982, 0.7406110713974553, 0.7500976785403125, 0.7595842856831696, 0.7690708928260268, 0.7785574999688839, 0.7824637499688839, 0.7863699999688839, 0.7902762499688839, 0.7941824999688839, 0.8019949999688839, 0.8137137499688839, 0.825432499968884, 0.844963749968884, 0.864494999968884, 0.884026249968884, 0.903557499968884, 0.923088749968884, 0.9426199999688841, 0.9621512499688841, 0.9922851785403126, 1.022419107111741, 1.05255303568317, 1.082686964254598, 1.1128208928260268, 1.142954821397456, 1.173088749968884, 1.214383392826027, 1.25567803568317, 1.296972678540313, 1.3382673213974559, 1.3795619642545989, 1.420856607111742, 1.462151249968884, 1.51739678568317, 1.5726423213974559, 1.6278878571117419, 1.6831333928260281, 1.738378928540313, 1.7936244642545989, 1.848869999968885, 1.925878928540313, 2.002887857111742, 2.079896785683171, 2.1569057142545986, 2.233914642826028, 2.310923571397457, 2.387932499968885, 2.457128928540313, 2.526325357111742, 2.5955217856831707, 2.6647182142546, 2.733914642826028, 2.803111071397457, 2.872307499968885, 2.9381557142546, 3.004003928540314, 3.069852142826028, 3.1357003571117428, 3.2015485713974567, 3.267396785683172, 3.333244999968886, 3.396861071397458, 3.4604771428260293, 3.5240932142546, 3.587709285683172, 3.6513253571117428, 3.714941428540315, 3.778557499968886, 3.799762857111743, 3.8209682142546, 3.842173571397457, 3.863378928540314, 3.884584285683171, 3.905789642826027, 3.9269949999688842, 3.930901249968884, 3.997307499968876, 4.0])

y_values_mv = np.array([-2.149425, -2.021906, -1.900015, -1.783753, -1.67312, -1.568115, -1.468739, -1.374991, -1.332516, -1.291012, -1.250478, -1.210915, -1.172323, -1.134702, -1.098051, -1.0832, -1.068501, -1.05395, -1.025293, -0.9833953, -0.9427724, -0.8781529, -0.8164742, -0.7577361, -0.7019387, -0.6490821, -0.5991661, -0.5521909, -0.4931582, -0.4377458, -0.385954, -0.3377825, -0.2932315, -0.2523009, -0.2149908, -0.1881227, -0.1626748, -0.1386472, -0.1160399, -0.09485276, -0.07508589, -0.05673927, -0.04842326, -0.04034997, -0.03251939, -0.02493152, -0.01758636, -0.01048392, -0.003624198, -0.0008585322, 0.001869788, 0.004561141, 0.007215904, 0.01241715, 0.01995378, 0.02718076, 0.03846925, 0.04906124, 0.05895672, 0.06815569, 0.07665815, 0.08446411, 0.09157355, 0.1013317, 0.1100654, 0.1177748, 0.1244598, 0.1301204, 0.1347566, 0.1383684, 0.1420968, 0.144932, 0.1468741, 0.147923, 0.1480789, 0.1473415, 0.1457111, 0.1428017, 0.1394779, 0.1357397, 0.1315871, 0.1270201, 0.1220387, 0.1166429, 0.1091808, 0.1019231, 0.09486995, 0.08802123, 0.08137697, 0.07493717, 0.06870183, 0.06353947, 0.0586587, 0.05405952, 0.04974192, 0.0457059, 0.04195147, 0.03847863, 0.03541913, 0.03254952, 0.02986978, 0.02737992, 0.02507993, 0.02296983, 0.02104961, 0.019333, 0.01772848, 0.01623606, 0.01485573, 0.01358749, 0.01243135, 0.01138729, 0.01105281, 0.01072709, 0.01041011, 0.01010189, 0.009802425, 0.009511712, 0.009229752, 0.009178586, 0.008348948, 0.008316821])

print(len(x_values_mv), len(y_values_mv))

x = np.arange(0.01, 4, 0.01)

122 122


plot the output of the GPR

In [1004]:
model = GPR(x_values, y_values)
data = plot_GPR(data_x=x_values, data_y=y_values, x=x, model=model, data_x_mv=x_values_mv, data_y_mv=y_values_mv)
fig4 = go.Figure(data)
fig4 = update_layout_of_graph(fig=fig4, title=f'GPR with length {model.cov_func.length}, sigma {model.cov_func.sigma}, alpha {cov_func.alpha}, beta {model.cov_func.beta} and noise {model.noise}')
fig4.show()
fig4.write_html('C:\\Users\\simon\\Desktop\\RLC_Current\\RLC_Current_aperiodic_0.75.html')

sigma memory =  0.2
length memory =  0.1


KeyboardInterrupt: 